In [21]:
import pandas as pd

In [22]:
df = pd.read_csv('data/06.csv')
df = df[df["LMP_TYPE"] == "LMP"]

df['INTERVALSTARTTIME_GMT'] = pd.to_datetime(df['INTERVALSTARTTIME_GMT'], utc=True)
df['INTERVALENDTIME_GMT'] = pd.to_datetime(df['INTERVALENDTIME_GMT'], utc=True)

# Step 2: Convert to California time
df['INTERVALSTARTTIME_CA'] = df['INTERVALSTARTTIME_GMT'].dt.tz_convert('America/Los_Angeles')
df['INTERVALENDTIME_CA'] = df['INTERVALENDTIME_GMT'].dt.tz_convert('America/Los_Angeles')

# Step 3: Remove timezone info (drop the -07:00)
df['INTERVALSTARTTIME_CA'] = df['INTERVALSTARTTIME_CA'].dt.tz_localize(None)
df['INTERVALENDTIME_CA'] = df['INTERVALENDTIME_CA'].dt.tz_localize(None)

# Step 4: Reorder so CA local times appear right after GMT or replace them
# Option A: Replace GMT columns entirely
df = df.drop(columns=['INTERVALSTARTTIME_GMT', 'INTERVALENDTIME_GMT'])  # optional
df = df.rename(columns={
    'INTERVALSTARTTIME_CA': 'INTERVALSTARTTIME',
    'INTERVALENDTIME_CA': 'INTERVALENDTIME'
})

df = df.drop(columns=['NODE_ID_XML', 'OPR_HR', 'NODE_ID', 'PNODE_RESMRID', 'GROUP', 'LMP_TYPE', 'POS', 'LMP_TYPE', 'MARKET_RUN_ID', 'OPR_INTERVAL', 'XML_DATA_ITEM', 'GRP_TYPE'])  # Drop extras
df = df.sort_values(by=['NODE', 'INTERVALSTARTTIME']).reset_index(drop=True)
df['Hour_Label'] = df['INTERVALSTARTTIME'].dt.strftime('%H:%M') + '-' + (df['INTERVALSTARTTIME'] + pd.Timedelta(hours=1)).dt.strftime('%H:%M')

df.head() 

,OPR_DT,NODE,MW,INTERVALSTARTTIME,INTERVALENDTIME,Hour_Label
0,2025-05-06,AFPR_1_TOT_GEN-APND,41.55134,2025-05-06 00:00:00,2025-05-06 01:00:00,00:00-01:00
1,2025-05-06,AFPR_1_TOT_GEN-APND,39.25263,2025-05-06 01:00:00,2025-05-06 02:00:00,01:00-02:00
2,2025-05-06,AFPR_1_TOT_GEN-APND,38.77800,2025-05-06 02:00:00,2025-05-06 03:00:00,02:00-03:00
3,2025-05-06,AFPR_1_TOT_GEN-APND,38.81647,2025-05-06 03:00:00,2025-05-06 04:00:00,03:00-04:00
4,2025-05-06,AFPR_1_TOT_GEN-APND,39.76272,2025-05-06 04:00:00,2025-05-06 05:00:00,04:00-05:00


In [23]:
df.to_csv("data/06pre.csv", index=False)